In [50]:
import numpy as np 
import pandas as pd

In [51]:
# telling numpy to not use scientific notation and displaying only 2 digits after decimal for numbers
np.set_printoptions(suppress = True, precision = 2)

In [52]:
# imported data also have # in dataset so had to changes comments = | b/c had issue with import.
lab_test_rawdata = np.genfromtxt("Lab-test-import-edit1.csv",
                              delimiter = ',',
                              comments="|", 
                              skip_header = 1)
lab_test_rawdata

array([[  44.  ,     nan,     nan, ...,     nan,     nan,     nan],
       [  45.  ,     nan,     nan, ...,  124.58,    0.  ,    0.  ],
       [  46.  ,     nan,     nan, ...,     nan,     nan,     nan],
       ...,
       [7394.  ,     nan,     nan, ...,    0.  ,    0.  ,    0.  ],
       [7395.  ,     nan,     nan, ...,    0.  ,    4.  ,    0.  ],
       [7396.  ,     nan,     nan, ...,    0.  ,    4.  ,    0.  ]])

In [53]:
#checking to see how many NAN values we have 
np.isnan(lab_test_rawdata).sum()

54401

In [54]:
#running this to see which cols contain all string values and which ones contain int.
temp_mean = np.nanmean(lab_test_rawdata, axis = 0)
temp_mean 

<ipython-input-54-4ba9b80e09f0>:2: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(lab_test_rawdata, axis = 0)


array([3676.07,     inf,     nan,     nan,     nan,     nan,  199.83,
          1.16,    4.92,    1.04,    5.21,   64.76,   23.39,    7.79])

In [55]:
#want to split data into strings cols and numeric cols 
#if col contains strings, mean will return nan and if .isnan 
#the argwhere tests if values are different from 0 since true (.isnan) is different from 0, this will return all the index values of the columns that contain strings. 

col_str = np.argwhere(np.isnan(temp_mean)).squeeze()
col_str

array([2, 3, 4, 5], dtype=int64)

In [56]:
#want to do the same thing for columns with numeric data for further analysis
col_numeric = np.argwhere(np.isnan(temp_mean)== False).squeeze()
col_numeric

array([ 0,  1,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64)

In [57]:
#then we need to re-import the data in two separate data sets
#used filling_val = 0 b/c with this particular dataset i know that if we have a missing value we didnt test sample so it should be represented as 0
lab_test_data_numeric = np.genfromtxt("Lab-test-import-edit1.csv", 
                                      delimiter = ',', 
                                      skip_header = 1, 
                                      autostrip = True, 
                                      usecols = col_numeric, 
                                      filling_values = 0, 
                                      comments= "|")
lab_test_data_numeric

array([[  44.  ,    0.  ,  204.  , ...,    0.  ,    0.  ,    0.  ],
       [  45.  ,    0.  ,  120.  , ...,  124.58,    0.  ,    0.  ],
       [  46.  ,    0.  ,  120.  , ...,    0.  ,    0.  ,    0.  ],
       ...,
       [7394.  ,    0.  ,  109.5 , ...,    0.  ,    0.  ,    0.  ],
       [7395.  ,    0.  ,  296.  , ...,    0.  ,    4.  ,    0.  ],
       [7396.  ,    0.  ,  344.  , ...,    0.  ,    4.  ,    0.  ]])

In [58]:
lab_test_data_str = np.genfromtxt("Lab-test-import-edit1.csv", 
                                      delimiter = ',', 
                                      skip_header = 1, 
                                      autostrip = True, 
                                      usecols = col_str, 
                                      comments= "|",
                                      dtype = np.str)
lab_test_data_str

<ipython-input-58-00f59c810cf3>:7: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str)


array([['Certification', 'No', 'Approved', 'Devin Muldoon'],
       ['Certification', 'Yes', 'Rejected', 'Devin Muldoon'],
       ['Certification', 'Yes', 'Rejected', 'Devin Muldoon'],
       ...,
       ['Audit', 'No', 'Approved', 'Lab Tech'],
       ['API Verification', 'No', 'Approved', 'Lab Tech'],
       ['API Verification', 'No', 'Approved', 'Lab Tech']], dtype='<U16')

In [59]:
#now need to import the headers so we don't lose track of what data is what
#skip_footer line ignores all rows after the header since we skipped header in import. 
full_header = np.genfromtxt("Lab-test-import-edit1.csv", 
                           delimiter = ',', 
                           skip_footer = lab_test_rawdata.shape[0],
                           autostrip = True, 
                           dtype = np.str)
full_header

<ipython-input-59-d0a467b7800c>:7: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.str)


array(['lab_id', 'Item', 'Submit_Type', 'Flagged', 'Approval',
       'Approved_By', 'API', 'Heated_Plate_Self', 'Heated_Plate_deltaT',
       'Heated_Plate_System', 'Air_flow', 'Air_Perm', 'Absorbency_0W',
       'Absorbency_2W'], dtype='<U19')

In [60]:
#now we need to separate out the headers
str_header, numeric_header = full_header[col_str], full_header[col_numeric]
numeric_header
str_header

array(['Submit_Type', 'Flagged', 'Approval', 'Approved_By'], dtype='<U19')

In [61]:
#now that we have separated the data, good time to set up a checkpoint so we don't lose work so far
def checkpoint(file_name, checkpoint_header, checkpoint_data):
            np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
            checkpoint_variable = np.load(file_name + ".npz")
            return(checkpoint_variable)

In [62]:
#assigning a new variable to the checkpoint function we just defined 
checkpoint_str = checkpoint("checkpoint-str", str_header, lab_test_data_str)
checkpoint_str['data']
checkpoint_str['header']

array(['Submit_Type', 'Flagged', 'Approval', 'Approved_By'], dtype='<U19')

In [63]:
checkpoint_numeric = checkpoint("checkpoint-numeric", numeric_header, lab_test_data_numeric)
checkpoint_numeric['data']
checkpoint_numeric['header']

array(['lab_id', 'Item', 'API', 'Heated_Plate_Self',
       'Heated_Plate_deltaT', 'Heated_Plate_System', 'Air_flow',
       'Air_Perm', 'Absorbency_0W', 'Absorbency_2W'], dtype='<U19')

In [64]:
#after examning data, i see that 'flagged' and 'approved_by' cols are redundant so i can drop them
str_header = np.delete(str_header,(1,3))
str_header

array(['Submit_Type', 'Approval'], dtype='<U19')

In [65]:
lab_test_data_str

array([['Certification', 'No', 'Approved', 'Devin Muldoon'],
       ['Certification', 'Yes', 'Rejected', 'Devin Muldoon'],
       ['Certification', 'Yes', 'Rejected', 'Devin Muldoon'],
       ...,
       ['Audit', 'No', 'Approved', 'Lab Tech'],
       ['API Verification', 'No', 'Approved', 'Lab Tech'],
       ['API Verification', 'No', 'Approved', 'Lab Tech']], dtype='<U16')

In [66]:
#now need to drop the actual data
lab_test_data_str = np.delete(lab_test_data_str,(1,3), axis = 1)

In [67]:
lab_test_data_str

array([['Certification', 'Approved'],
       ['Certification', 'Rejected'],
       ['Certification', 'Rejected'],
       ...,
       ['Audit', 'Approved'],
       ['API Verification', 'Approved'],
       ['API Verification', 'Approved']], dtype='<U16')

In [68]:
#need to group api verification and certification together, audit and resubmit are in separate groups
#0 = 'Resubmit', 1 ='Api ver' and 'cert', 2='audit'
np.unique(lab_test_data_str[:,0])

array(['API Verification', 'Audit', 'Certification', 'Resubmit'],
      dtype='<U16')

In [69]:
ver_cert = np.array(['API Verification','Certification'])

In [70]:
lab_test_data_str[:,0] = np.where(np.isin(lab_test_data_str[:,0], 'Resubmit'), 0, lab_test_data_str[:,0])
lab_test_data_str[:,0] = np.where(np.isin(lab_test_data_str[:,0], ver_cert),1, lab_test_data_str[:,0])
lab_test_data_str[:,0] = np.where(np.isin(lab_test_data_str[:,0], 'Audit'), 2, lab_test_data_str[:,0])

In [71]:
np.unique(lab_test_data_str[:,0])

array(['0', '1', '2'], dtype='<U16')

In [72]:
np.unique(lab_test_data_str[:,1])

array(['', 'Approved', 'Pending Approval', 'Rejected'], dtype='<U16')

In [73]:
#Since numbers are easier for analysis, we are going to change all status to either good/bad. 
#Good = 'Approved', 'Pending Approval' , Bad = 'Rejected' or blank
bad_status = np.array(['','Rejected'])

In [74]:
#generally associate 1 with good and 0 with bad so i will now make apprpriate changes for later analysis
lab_test_data_str[:,1] = np.where(np.isin(lab_test_data_str[:,1], bad_status),0,1)

In [75]:
np.unique(lab_test_data_str[:,1])

array(['0', '1'], dtype='<U16')

In [76]:
lab_test_data_str

array([['1', '1'],
       ['1', '0'],
       ['1', '0'],
       ...,
       ['2', '1'],
       ['1', '1'],
       ['1', '1']], dtype='<U16')

In [77]:
#now that we have all numbers we need to convert them to acutal numbers instead of strings 
lab_test_data_str = lab_test_data_str.astype(np.int8)
lab_test_data_str

array([[1, 1],
       [1, 0],
       [1, 0],
       ...,
       [2, 1],
       [1, 1],
       [1, 1]], dtype=int8)

In [78]:
#from reviewing data, i think that lab_id col is more important than the item col and both represent the same thing
numeric_header = np.delete(numeric_header, 1)

In [79]:
numeric_header

array(['lab_id', 'API', 'Heated_Plate_Self', 'Heated_Plate_deltaT',
       'Heated_Plate_System', 'Air_flow', 'Air_Perm', 'Absorbency_0W',
       'Absorbency_2W'], dtype='<U19')

In [80]:
lab_test_data_numeric

array([[  44.  ,    0.  ,  204.  , ...,    0.  ,    0.  ,    0.  ],
       [  45.  ,    0.  ,  120.  , ...,  124.58,    0.  ,    0.  ],
       [  46.  ,    0.  ,  120.  , ...,    0.  ,    0.  ,    0.  ],
       ...,
       [7394.  ,    0.  ,  109.5 , ...,    0.  ,    0.  ,    0.  ],
       [7395.  ,    0.  ,  296.  , ...,    0.  ,    4.  ,    0.  ],
       [7396.  ,    0.  ,  344.  , ...,    0.  ,    4.  ,    0.  ]])

In [81]:
lab_test_data_numeric = np.delete(lab_test_data_numeric, 1, axis = 1)

In [82]:
lab_test_data_numeric 

array([[  44.  ,  204.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [  45.  ,  120.  ,    0.87, ...,  124.58,    0.  ,    0.  ],
       [  46.  ,  120.  ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       ...,
       [7394.  ,  109.5 ,    0.  , ...,    0.  ,    0.  ,    0.  ],
       [7395.  ,  296.  ,    0.  , ...,    0.  ,    4.  ,    0.  ],
       [7396.  ,  344.  ,    0.  , ...,    0.  ,    4.  ,    0.  ]])

In [83]:
#here i want to double check that there are no missing values in the data set. 0 confirms this is the case. 
np.isnan(lab_test_data_numeric).sum()

0

In [84]:
#need to double check both arrays have compatible shapes in order to combine. 
lab_test_data_numeric.shape

(6773, 9)

In [85]:
lab_test_data_str.shape

(6773, 2)

In [86]:
#need to combine to two datasets back into one now that it has been cleaned and preprocessed. 
lab_test_data_cleaned = np.hstack((lab_test_data_numeric, lab_test_data_str))
lab_test_data_cleaned

array([[  44.  ,  204.  ,    0.  , ...,    0.  ,    1.  ,    1.  ],
       [  45.  ,  120.  ,    0.87, ...,    0.  ,    1.  ,    0.  ],
       [  46.  ,  120.  ,    0.  , ...,    0.  ,    1.  ,    0.  ],
       ...,
       [7394.  ,  109.5 ,    0.  , ...,    0.  ,    2.  ,    1.  ],
       [7395.  ,  296.  ,    0.  , ...,    0.  ,    1.  ,    1.  ],
       [7396.  ,  344.  ,    0.  , ...,    0.  ,    1.  ,    1.  ]])

In [87]:
#here we can check to make sure that we successfully added the two arrays columns on 
lab_test_data_cleaned.shape

(6773, 11)

In [88]:
np.isnan(lab_test_data_cleaned).sum()

0

In [89]:
#combining the headers then adding to the complete dataset 
full_header = np.concatenate((numeric_header, str_header))
full_header

array(['lab_id', 'API', 'Heated_Plate_Self', 'Heated_Plate_deltaT',
       'Heated_Plate_System', 'Air_flow', 'Air_Perm', 'Absorbency_0W',
       'Absorbency_2W', 'Submit_Type', 'Approval'], dtype='<U19')

In [92]:
#adding headers to full dataset 
full_data_cleaned = np.vstack((full_header, lab_test_data_cleaned))
full_data_cleaned

array([['lab_id', 'API', 'Heated_Plate_Self', ..., 'Absorbency_2W',
        'Submit_Type', 'Approval'],
       ['44.0', '204.0', '0.0', ..., '0.0', '1.0', '1.0'],
       ['45.0', '120.0', '0.87', ..., '0.0', '1.0', '0.0'],
       ...,
       ['7394.0', '109.5', '0.0', ..., '0.0', '2.0', '1.0'],
       ['7395.0', '296.0', '0.0', ..., '0.0', '1.0', '1.0'],
       ['7396.0', '344.0', '0.0', ..., '0.0', '1.0', '1.0']], dtype='<U32')

In [93]:
#now we need to save the cleaned dataset 
np.savetxt("Full-lab-data-cleaned.csv",
          full_data_cleaned, 
          fmt= "%s", 
          delimiter = ',')